TODO: 
1) Fix Country aliasing
2) Decide between GDPPC and median income/consumption
3) Fix Regioning

In [11]:
import numpy as np
import pandas as pd

In [12]:
parse ="""
Algeria
Angola
Argentina
Australia
Austria
Azerbaijan
Bahrain
Bangladesh
Belarus
Belgium
Benin
Bolivia
Botswana
Brazil
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Chad
Chile
China
Colombia
Congo (Dem. Rep.)
Costa Rica
Côte d'Ivoire
Czech Republic
Denmark
Dominican Rep.
Ecuador
Egypt
El Salvador
Ethiopia
Finland
France
Germany
Ghana
Greece
Guatemala
Guinea
Haiti
Honduras
Hungary
India
Indonesia
Ireland
Israel
Italy
Japan
Jordan
Kazakhstan
Kenya
Kuwait
Laos
Madagascar
Malawi
Malaysia
Mali
Mexico
Morocco
Mozambique
Myanmar
Nepal
Netherlands
New Zealand
Nicaragua
Niger
Nigeria
Norway
Oman
Pakistan
Panama
Paraguay
Peru
Philippines
Poland
Portugal
Qatar
Romania
Russia
Rwanda
Saudi Arabia
Senegal
Serbia
Sierra Leone
Singapore
Slovakia
South Africa
South Korea
Spain
Sri Lanka
Sudan
Sweden
Switzerland
Syria
Tajikistan
Tanzania
Thailand
Togo
Tunisia
Turkey
Uganda
Ukraine
United Arab Emirates
United Kingdom
United States
Uruguay
Uzbekistan
Venezuela
Vietnam
Yemen
Zambia
"""
df_cnt = pd.DataFrame({"Country": [s for s in parse.split("\n") if not s==""]})
df_cnt#14,16,19,21

,Country
0,Algeria
1,Angola
2,Argentina
3,Australia
4,Austria
...,...
108,Uzbekistan
109,Venezuela
110,Vietnam
111,Yemen


In [13]:
df_gini = pd.read_csv("GINI_ALLYR.csv").drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
df_median = pd.read_csv("MEDIAN_PPP.csv")[['country_name','region_name','median','reporting_year','reporting_level']]
df_median.rename(columns={'country_name':'Country','region_name':'Region','reporting_year':'year'},inplace=True)
df_imports = pd.read_csv("IMPORT.csv")
df_pop = pd.read_csv("POP.csv").drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
df_exports = pd.read_csv("EXPORT.csv")
df_gdppc = pd.read_csv("GDPPC.csv").drop(['Country Code','Indicator Name','Indicator Code'],axis=1)


In [20]:
#time serieses --> export, fpi,gdpcc,gini,import,pop
#row time series --> median
#categorical --> region

#print(df_median.head(3))
df_gini_c = pd.merge(df_cnt, df_gini,how="inner",left_on="Country",right_on="Country Name").drop("Country Name",axis=1)
df_median_c = pd.merge(df_cnt, df_median,how="inner",left_on="Country",right_on="Country")#.drop("country_name",axis=1)
df_imports_c = pd.merge(df_cnt, df_imports,how="inner",on="Country").drop(["Indicator",'Product Group','Trade Flow'],axis=1)
df_exports_c = pd.merge(df_cnt, df_exports,how="inner",on="Country").drop(["Indicator",'Product Group','Trade Flow'],axis=1)
df_pop_c = pd.merge(df_cnt, df_pop,how="inner",left_on="Country",right_on="Country Name").drop("Country Name",axis=1)
df_gdppc_c = pd.merge(df_cnt, df_gdppc,how="inner",left_on="Country",right_on="Country Name").drop("Country Name",axis=1)
#deal with median data national level only
national_only = (df_median_c["reporting_level"] == "national")
df_median_cc = df_median_c[national_only]
#print(df_median_cc)
df_na = df_median_cc[df_median_cc["median"].isnull()]
#print(df_na)
for index,row in df_na.iterrows():
    ch_ur = df_median_c.loc[(df_median_c["Country"]==row["Country"])&(df_median_c["year"]==row["year"])&(df_median_c["reporting_level"]=="urban")]
    ch_ru = df_median_c.loc[(df_median_c["Country"]==row["Country"])&(df_median_c["year"]==row["year"])&(df_median_c["reporting_level"]=="rural")]
    if len(ch_ur)>0:
        df_median_cc.loc[index,"median"] = ch_ur.iloc[0]["median"]
        #print("Amend\n", df_median_cc.loc[index])
    elif len(ch_ru)>0:
        df_median_cc.loc[index,"median"] = ch_ru.iloc[0]["median"]
        #print("Amend\n", df_median_cc.loc[index])
    else:
        df_median_cc.drop(index=index)
df_median_cc.drop(columns="reporting_level",inplace=True)


Dropping 504
Dropping 648
Dropping 1315


/tmp/ipykernel_84204/2341780323.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_median_cc.drop(columns="reporting_level",inplace=True)


In [15]:

df_gini_cc = df_gini_c.melt(id_vars="Country",var_name="year",value_name="gini").dropna().reset_index(drop=True)
df_exports_cc = df_exports_c.melt(id_vars="Country",var_name="year",value_name="exports").dropna().reset_index(drop=True)
df_imports_cc = df_imports_c.melt(id_vars="Country",var_name="year",value_name="imports").dropna().reset_index(drop=True)
df_pop_cc = df_pop_c.melt(id_vars="Country",var_name="year",value_name="pop").dropna().reset_index(drop=True)
df_gdppc_cc = df_gdppc_c.melt(id_vars="Country",var_name="year",value_name="gdppc").dropna().reset_index(drop=True)
feature_set = [df_gini_cc,df_exports_cc,df_imports_cc,df_pop_cc,df_gdppc_cc,df_median_cc]



In [16]:
#deal with targets
tgt_sub_l = []
for n in range(10):
    tgt = pd.read_excel(f"{2012+n}_GFSI_Data_Table_Scores.xlsx","Sheet1",skiprows=5,usecols="B:DP")
    keepdata = ['FOOD SECURITY ENVIRONMENT','1) AFFORDABILITY','2) AVAILABILITY','3) QUALITY AND SAFETY','4) SUSTAINABILITY AND ADAPTATION']
    tgt.rename( columns={'Unnamed: 1':'TargetVal'}, inplace=True )
    chk = tgt["TargetVal"].isin(keepdata)
    tgt_sub = tgt[chk].drop(['Weight','Average (mean)','Average (median)','Minimum','Maximum'],axis=1)
    tgt_sub_l.append(tgt_sub.T) #transpose to get by country
    tgt_sub_l[-1]["year"] = np.ones((tgt_sub_l[-1].shape[0],1),dtype="int32")*int(2012+n)
    tgt_sub_l[-1] = tgt_sub_l[-1].rename(columns={0:'Security',1:"Affordability",7:'Avail',17:'Quality',23:'Sustain'})#,'2) AVAILABILITY':"Availability",'3) QUALITY AND SAFETY':"Quality",'4) SUSTAINABILITY AND ADAPTATION':"Sustainability"},inplace=True)
    tgt_sub_l[-1].drop(index=tgt_sub_l[-1].index[0],inplace=True)
merged_targets = pd.concat(tgt_sub_l)
#merged_targets["Country"]=list(merged_targets.index)
merged_targets.reset_index(inplace=True)
merged_targets.rename(columns={'index':"Country"},inplace = True)
#merged_targets

In [17]:
#find all intersection
master_clean = merged_targets.copy()
for feature in feature_set:
    feature['year'] = feature['year'].astype("int64")
    master_clean = pd.merge(master_clean, feature,how="inner",on=['Country','year'])
master_clean.drop_duplicates(inplace=True)
master_clean.to_csv('Intersecting_clean.csv',index=False)


/tmp/ipykernel_84204/3889571712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature['year'] = feature['year'].astype("int64")


In [18]:
master_clean

,Country,Security,Affordability,Avail,Quality,Sustain,year,gini,exports,imports,pop,gdppc,Region,median
0,Austria,74.4,87.0,67.5,87.2,52.3,2012,30.5,5706086.36,6.686922e+06,8429991.0,48564.920,Other High Income Countries,52.697230
1,Belarus,60.2,65.5,51.8,74.5,48.2,2012,26.5,1387263.07,4.396182e+05,9446836.0,6953.217,Europe & Central Asia,14.584608
2,Belgium,73.6,88.9,65.8,78.1,57.6,2012,27.5,16951434.0,1.780664e+07,11106932.0,44670.560,Other High Income Countries,48.920853
3,Bolivia,52.8,57.9,54.1,52.6,44.7,2012,46.6,394016.576,7.967270e+05,10569697.0,2562.467,Latin America & Caribbean,13.299046
4,Brazil,63.8,72.6,61.3,74.1,44.7,2012,53.4,3785472.79,3.086821e+07,199977707.0,12327.520,Latin America & Caribbean,11.778717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,Peru,73.1,86.1,61.4,74.3,67.5,2021,40.2,2134258.51,4.148974e+06,33715471.0,6635.464,Latin America & Caribbean,9.126669
442,Philippines,59.6,74.6,54.8,64.6,40.0,2021,40.7,7203271.95,2.883677e+06,113880328.0,3460.539,East Asia & Pacific,4.976315
443,Thailand,62.1,83.2,56.7,50.9,51.4,2021,35.1,8273036.68,1.910214e+07,71601103.0,7060.898,East Asia & Pacific,12.938363
444,United States,78.7,87.6,67.3,88.8,69.4,2021,39.8,106698855.0,5.335824e+07,332031554.0,70219.470,Other High Income Countries,66.645184


In [19]:
list(master_clean.columns)

['Country',
 'Security',
 'Affordability',
 'Avail',
 'Quality',
 'Sustain',
 'year',
 'gini',
 'exports',
 'imports',
 'pop',
 'gdppc',
 'Region',
 'median']